# Spatial Queries with Geopandas

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

In [ ]:
import geoplot as gplt

In [ ]:
import geoplot.crs as gcrs

In [ ]:
gdf = gpd.read_file("geodata/packages/natural_earth_vector.gpkg", layer='ne_10m_admin_0_countries')

## Queries with Polygons

Let's create polygon covering some part of Europe:

In [ ]:
polygon = Polygon([(-10, 35), (-10, 60), (40, 60), (40, 35)]) # this is our polygon


### Display the polygon and our vector file

To understand what we're doing let's display our data using Geoplot

In [ ]:
polygon_gseries = gpd.GeoSeries(polygon)
polygon_gdf = gpd.GeoDataFrame(geometry=polygon_gseries,crs="EPSG:4326") 

ax = gplt.polyplot(gdf, projection=gcrs.PlateCarree(), figsize=(10, 10))
gplt.polyplot(polygon_gdf, ax=ax, facecolor='none', edgecolor='red', linewidth=2)
ax.set_xlim([-180, 180])
ax.set_ylim([-90, 90]);

### Within

In [ ]:
gdf_within_polygon = gdf[gdf.within(polygon)]

In [ ]:
ax = gplt.polyplot(gdf, projection=gcrs.PlateCarree(), figsize=(10, 10))
gplt.polyplot(polygon_gdf, ax=ax, facecolor='none', edgecolor='red', linewidth=2)
gplt.polyplot(gdf_within_polygon, ax=ax, edgecolor="green")
ax.set_xlim([-60, 60])
ax.set_ylim([20, 80]);

### Contains

In spatial analysis contains and within are two different spatial relationships:

**contains**: Geometry A contains Geometry B if and only if all points of B are also points of A, and the interiors of A and B have at least one point in common. In other words, A contains B if B is completely inside A, including its boundary. In the context of polygons, Polygon A contains Polygon B if every vertex of B is inside A, and their interiors have at least one point in common.

**within**: Geometry A is within Geometry B if and only if all points of A are also points of B, and the interiors of A and B have at least one point in common. In other words, A is within B if A is completely inside B, including its boundary. In the context of polygons, Polygon A is within Polygon B if every vertex of A is inside B, and their interiors have at least one point in common.



In [ ]:
gdf_contains_polygon = gdf[gdf.contains(polygon)]

In [ ]:
ax = gplt.polyplot(gdf, projection=gcrs.PlateCarree(), figsize=(10, 10))
gplt.polyplot(polygon_gdf, ax=ax, facecolor='none', edgecolor='red', linewidth=2)
gplt.polyplot(gdf_contains_polygon, ax=ax, edgecolor="green")
ax.set_xlim([-60, 60])
ax.set_ylim([20, 80]);

The Polygon is not within any other polygon, lets create a polygon which lies inside of France:

In [ ]:
polygon2 = Polygon([(4.75, 45.75), (5.25, 45.75), (5.25, 46.25), (4.75, 46.25), (4.75, 45.75)])


#### Let's display this new polygon

The polygon is a small polygon **within** France. Remember, the query is to find all Polyons (countries) that **contain** this polygon.

In [ ]:
polygon2_gseries = gpd.GeoSeries(polygon2)
polygon2_gdf = gpd.GeoDataFrame(geometry=polygon2_gseries,crs="EPSG:4326") 

ax = gplt.polyplot(gdf, projection=gcrs.PlateCarree(), figsize=(10, 10))
gplt.polyplot(polygon2_gdf, ax=ax, facecolor='none', edgecolor='red', linewidth=1)
ax.set_xlim([-30, 30])
ax.set_ylim([40, 60]);

#### Do the "within" query with our new polygon2

In [ ]:
gdf_contains_polygon2 = gdf[gdf.contains(polygon2)]

In [ ]:
ax = gplt.polyplot(gdf, projection=gcrs.PlateCarree(), figsize=(10, 10))
gplt.polyplot(polygon2_gdf, ax=ax, facecolor='none', edgecolor='red', linewidth=1)
gplt.polyplot(gdf_contains_polygon2, ax=ax, edgecolor="green")
ax.set_xlim([-30, 30])
ax.set_ylim([40, 60]);

#### Of course we can also output the Geodataframe of the result

In [ ]:
gdf_contains_polygon2

### Intersects

Let's use our first polygon again for the intersection

In [ ]:
gdf_intersects_polygon = gdf[gdf.intersects(polygon)]

In [ ]:
ax = gplt.polyplot(gdf, projection=gcrs.PlateCarree(), figsize=(10, 10))
gplt.polyplot(polygon_gdf, ax=ax, facecolor='none', edgecolor='red', linewidth=2)
gplt.polyplot(gdf_intersects_polygon, ax=ax, edgecolor="green")
ax.set_xlim([-60, 60])
ax.set_ylim([20, 80]);

## Distances

Let's calculate the distances from the City of Basel, Switzerland to all countries in the dataset:

In [ ]:
gdf = gpd.read_file("geodata/packages/natural_earth_vector.gpkg", layer='ne_10m_admin_0_countries')

In [ ]:
from shapely.geometry import Point

basel = Point(7.5886, 47.5596)

To calculate distances, it's essential to use a suitable Coordinate Reference System (CRS) that allows for distance calculations in meters. 

One such CRS is the "World Equidistant Cylindrical" (EPSG:4087). 

Let's convert the original CRS of the GeoDataFrame and our Point to EPSG:4087:

In [ ]:
gdf_meters = gdf.to_crs('EPSG:4087')
basel_meters = gpd.GeoSeries(basel, crs='EPSG:4326').to_crs('EPSG:4087')
basel_meters_gdf = gpd.GeoDataFrame([{'name': 'Basel'}], geometry=basel_meters)

In [ ]:
basel_meters_gdf

In [ ]:
distances = gdf_meters.geometry.distance(basel_meters[0])

In [ ]:
distances

In [ ]:
gdf['distance_to_basel'] = distances / 1000  # save distance in km

In [ ]:
gdf.head(5)

In [ ]:
gdf.sort_values(by="distance_to_basel", ascending=True)